# Enefit - Quick EDA with Dash
[![Screenshot-2023-12-03-at-15-19-31-Enefit-Predict-Energy-Behavior-of-Prosumers-Kaggle.png](https://i.postimg.cc/2jLKqjXb/Screenshot-2023-12-03-at-15-19-31-Enefit-Predict-Energy-Behavior-of-Prosumers-Kaggle.png)](https://postimg.cc/3W7BVhhY)

## About this Competition
The energy imbalance problem indicates there exists imbalance between the **production and consumption** of energy. That is, the generated electricity mismatches the demanding side (*e.g.,* unexpected energy consumption behavior). In this competition, energy consumption/production patterns of prosumers are given with the auxiliary features (*e.g.,* weather data, gas and electricity prices, the installed PV capacity), competitors are challenged to predict **electricity produced and consumed by Estonian energy customers** who have installed solar panels, which can be formulated as a time series **regression** problem.

## About this Notebook
In this kernel, I will start from raw file exploration. Then, a time series data dashboard is implemented for interactive data exploration. Also, a naive baseline will be provided to see how far we can reach with inspirations from data exploration.

<a id="toc"></a>
## Table of Contents
* [1. Data Appearance](#data_appearance)
    * *[Our Main Target](#tgt)*
    * *[All about Weather](#wth)*
* [2. Towards Spatio-Temporal Analysis with Dash](#st_dash)
    * *[Load Necessary Data](#st_load_data)*
    * *[Generate Interactive Options](#st_gen_opts)*
    * *[Define Dashboard Layout](#st_layout)*
    * *[Define Callbacks](#st_cbs)*
    * *[Start Exploration](#st_start)* 
* [3. Baseline Model with Local CV](#baseline)
* [Reference](#ref)

## Import Packages

In [ ]:
!pip install dash-bootstrap-components
!pip uninstall polars -y
!pip install polars==0.19.16
!pip install pyngrok

In [ ]:
import json
import math
import pickle
from pathlib import Path
from typing import Dict, List, Optional

import plotly
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
import plotly.express as px
import dash_bootstrap_components as dbc
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
from pyngrok import ngrok
from dash import Dash, html, dcc, callback, Output, Input

pl.Config.set_tbl_hide_dataframe_shape(True)
init_notebook_mode(connected=True)

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
auth_token = user_secrets.get_secret("ngrok")
ngrok.set_auth_token(auth_token)
tunnel = ngrok.connect(8050)

## Define Paths, Metadata and Utilities

In [ ]:
# Paths
RAW_DATA_PATH = Path("/kaggle/input/predict-energy-behavior-of-prosumers")
PROC_DATA_PATH = Path("/kaggle/input/enefit-eda/enefit_eda")

# Metadata
UNIT_ID_COL = "prediction_unit_id"
TGT_PK_COLS = ["county", "is_business", "product_type"]
COORD_COL2ABBR = {"latitude": "lat", "longitude": "lon"}

In [ ]:
def _summarize(
    df: pl.DataFrame,
    file_name: Optional[str] = None,
    n_rows_to_display: Optional[int] = 5,
) -> None:
    """Summarize DataFrame.

    Args:
        df: input data
        file_name: name of the input file
        n_rows_to_display: number of rows to display
    """
    file_name = "Data" if file_name is None else file_name

    # Derive NaN ratio for each column
    nan_ratio = df.null_count() / len(df) * 100

    # Print out summarized information
    print(f"===== Summary of {file_name} =====")
    display(df.head(n_rows_to_display))
    print(f"Shape: {df.shape}")
    print("NaN ratio:")
    display(nan_ratio)

<a id="data_appearance"></a>
## 1. Data Appearance
<a id="tgt"></a>
### *Our Main Target*
[**<span style="color:#FEF1FE; background-color:#535d70;border-radius: 5px; padding: 2px">Go to Table of Content</span>**](#toc)

First, let's take a look at the main training set. It contains the **electricity consumption and production patterns** of prosumers, which can be determined by a indicator `is_consumption`. Also, the primary key is a composite key `(county, is_business, product_type)`, reduced to a single column `prediction_unit_id`.<br>
As can be observed, there exist **missing values** in `target` column, which is our predicting target.

In [ ]:
# Load train data
train = pl.read_csv(RAW_DATA_PATH / "train.csv", try_parse_dates=True)

_summarize(train, "train.csv", 1)
assert len(train.unique(subset=TGT_PK_COLS+[UNIT_ID_COL]))== len(set(train[UNIT_ID_COL]))

Next, let's plot and glimpse some randomly sampled consumption and production sequences jointly to observe their interaction. My intuition is that prosumers tend to consume more electricity when they produced more beforehand. However, the following plots seem not to support this assumption.<br>
Some interesting observations are summarized as follows:
1. There exist **periodicities** with different granularities (*e.g.,* daily, weekly, monthly).
2. There exist **breakpoints**, which are missing values.
3. There exists no negative target value.

In [ ]:
def _show_targets(df: pl.DataFrame, n_uids: int = 6) -> None:
    """Randomly sample and plot target sequences."""
    n_cols_per_row = 3
    colors = {0: "rgba(8, 209, 59, 0.5)", 1: "rgba(245, 10, 10, 0.7)"}
    sampled_ids = np.random.choice(df[UNIT_ID_COL].unique(), size=n_uids, replace=False)
    
    fig = make_subplots(rows=math.ceil(n_uids / n_cols_per_row), cols=n_cols_per_row)
    for i, uid in enumerate(sampled_ids):
        train_uid = (
            train
            .filter((pl.col(UNIT_ID_COL) == uid))
            .sort("datetime")
        )
        for is_cons in [0, 1]:
            train_uid_ = train_uid.filter(pl.col("is_consumption") == is_cons)
            row, col = i//n_cols_per_row + 1, i%n_cols_per_row + 1
            fig.add_trace(
                go.Scatter(
                    x=train_uid_["datetime"],
                    y=train_uid_["target"], 
                    name="Consumption" if is_cons else "Production",
                    line={"color": colors[is_cons]},
                    legendgroup=i,
                    legendgrouptitle_text=f"Plot ({row}, {col}) - UID {uid}"
                ),
                row=row, 
                col=col
            )
    
    fig.update_layout(
        title_text=f"Sampled Electricity Sequences",
        margin=dict(l=12, r=12, t=50, b=50)
    )
    fig.show()

In [ ]:
_show_targets(train, n_uids=6)
assert (train["target"] >= 0).all()

Diving into primary keys, we can find that,
1. Prosumers aren't equally distributed among different counties in Estonia.
2. \> 50% of prosumers are businesses.
3. `"Spot"` is the major contract type, followed by `"Fixed"`.

In [ ]:
with open(RAW_DATA_PATH / "county_id_to_name_map.json", "r") as f:
    county_id2name = json.load(f)
pks = train[TGT_PK_COLS].unique()

fig = make_subplots(
    rows=1, cols=3, 
    column_widths=[0.6, 0.2, 0.2], 
    specs=[[{"type": "xy"}, {"type": "domain"}, {"type": "domain"}]],
    subplot_titles=TGT_PK_COLS
)
# County ratio
county_ratio = pks["county"].value_counts()
county_ratio = county_ratio.with_columns(
    (pl.col("counts") / len(pks) * 100).alias("ratio"),
    (pl.col("county").cast(pl.Utf8).replace(county_id2name).alias("county_name"))
)
fig.add_trace(go.Bar(x=county_ratio["county_name"], y=county_ratio["ratio"]), row=1, col=1)
# Business or not
business_cnt = pks["is_business"].value_counts()
fig.add_trace(go.Pie(labels=business_cnt["is_business"], values=business_cnt["counts"], hoverinfo="label+percent"), row=1, col=2)
# Product types
prod_type_cnt = pks["product_type"].value_counts()
prod_type_cnt = prod_type_cnt.with_columns(
    (pl.col("product_type").replace({0: "Combined", 1: "Fixed", 2: "General service", 3: "Spot"}).alias("product_name"))
)
fig.add_trace(go.Pie(labels=prod_type_cnt["product_name"], values=prod_type_cnt["counts"], hoverinfo="label+percent"), row=1, col=3)
fig.update_layout(
    title_text="Primary Keys Individual Distribution",
    margin=dict(l=12, r=12, t=75, b=50),
    showlegend=False
)
fig.show()

<a id="wth"></a>
### *All about Weather*
[**<span style="color:#FEF1FE; background-color:#535d70;border-radius: 5px; padding: 2px">Go to Table of Content</span>**](#toc)

Energy generation is subject to different sources, from which the electricity is produced. Renewable sources (*e.g.,* solar) are also somewhat subject to weather condition (*e.g.,* rainfall and solar radiation). Hence, analyzing target patterns with weather data may provide hidden clues facilitating advanced feature engineering and modeling.<br>
There are two weather information provided, namely the **historical and forecast** ones. We should note that there exists missing data in forecast weather data, which can be interpolated based on different techniques (*e.g.,* interpolation from nearest neighbor station).

In [ ]:
h_wth = pl.read_csv(RAW_DATA_PATH / "historical_weather.csv", try_parse_dates=True)
f_wth = pl.read_csv(RAW_DATA_PATH / "forecast_weather.csv", try_parse_dates=True)
_summarize(h_wth, "historical_weather.csv", 1)
_summarize(f_wth, "forecast_weather.csv", 1)

[![Screenshot-2023-12-03-at-16-15-10-Google.png](https://i.postimg.cc/W3G2mR20/Screenshot-2023-12-03-at-16-15-10-Google.png)](https://postimg.cc/62QJwm4Q)

To merge target sequences with weather data, the naive method is to consider **when (`datetime`/`data_block_id`) and where (`county`/`lat`/`lon`)** the weather data is recorded and use this information as the key to combine them. Before merging, we need to first align the timeline of information that is available at forecast time, which can help prevent errors during test phase.<br>

The figure above illustrates the available weather data (other data can be confirmed in the similar way) during test phase. Also, as describe in the [data tab](https://www.kaggle.com/competitions/predict-energy-behavior-of-prosumers/data), forecast is made at **11:00 a.m. each morning**. Suppose we're making prediction for period from 00:00 a.m. to 11:00 p.m. on May 28, 2023 (as shown in the red period above), we have access to historical weather data in the blue period and forecast one in the green period. Moreover, you can cache any past data if you want to use them.<br>

Special thanks to valuable discussions and hard works on data block alignment and `datetime` fixing, which are listed as follows,
1. [Timeline of availability of the data and data_block_id](https://www.kaggle.com/competitions/predict-energy-behavior-of-prosumers/discussion/455833) by @jeanbaptistescellier
2. [Is the time zone the same for all data?](https://www.kaggle.com/competitions/predict-energy-behavior-of-prosumers/discussion/454145) by @sooyoungher
3. [Enefit|Polars-preprocessing](https://www.kaggle.com/code/michaelo/enefit-polars-preprocessing/notebook) by @michaelo

In [ ]:
def _cal_wth_local_mean(df_wth: pl.DataFrame, gp_keys: List[str]) -> pl.DataFrame:
    """Calculate county-level local weather stats.
    
    Only mean is derived now.

    Args:
        df_wth: weather data
        gp_keys: groupby keys

    Returns:
        df_wth_local_stats: county-level local weather stats
    """
    df_wth = df_wth.with_columns(*CAST_COORD)
    df_wth_local_stats = (
        df_wth
        .join(station_latlon_county_map, on=list(COORD_COL2ABBR.values()), how="left")
        .filter(pl.col("county").is_not_null())
        .group_by(gp_keys).mean()
        .select(*gp_keys, pl.exclude(gp_keys).name.suffix("_local_mean"))
    )

    return df_wth_local_stats

Let's load weather station positions which map each weather station to one of the county based on its coordinates (*i.e.,* `latitude` and `longitude` pair).

Special thanks to the following excellent works,
1. [mapping locations and county codes](https://www.kaggle.com/code/fabiendaniel/mapping-locations-and-county-codes) by @fabiendaniel
2. [fabiendaniels-mapping-locations-and-county-codes](https://www.kaggle.com/datasets/michaelo/fabiendaniels-mapping-locations-and-county-codes) by @michaelo

In [ ]:
station_latlon_county_map = pl.read_csv(PROC_DATA_PATH / "county_lat_lon.csv")

CAST_COORD = [pl.col("lat").cast(pl.Float32), pl.col("lon").cast(pl.Float32)]
station_latlon_county_map = (
    station_latlon_county_map
    .drop("")
    .rename(COORD_COL2ABBR)
    .with_columns(*CAST_COORD)
)

Then, let's process the historical weather data first. To align the timeline, the weather data can be shifted based on a specific period. Following the previous work, I take the time gap, 1 day plus 13 hours, as the period for shifting.

> Considering periodicity, it's possible to find a better shifting amount.

In [ ]:
h_wth = h_wth.with_columns(pl.col("datetime") + pl.duration(days=1, hours=13))

h_wth = h_wth.rename(COORD_COL2ABBR)
gp_keys_h = ["datetime", "county"]
h_wth_local_stats = _cal_wth_local_mean(h_wth, gp_keys=gp_keys_h)
_summarize(h_wth_local_stats, "Historical Weather Local Stats", 1)

As for forecast weather data, the main issue is the time zone mismatch. I also try to fix it in the same manner as the previous work.

In [ ]:
f_wth = f_wth.rename(COORD_COL2ABBR)
gp_keys_f = ["origin_datetime", "hours_ahead", "data_block_id", "forecast_datetime"] + ["county"]
f_wth_local_stats = _cal_wth_local_mean(f_wth, gp_keys_f)

f_wth_local_stats = (
    f_wth_local_stats
    .with_columns([ 
        pl.col("forecast_datetime")
        .dt.convert_time_zone("Europe/Bucharest").alias("datetime")
        .dt.replace_time_zone(None).cast(pl.Datetime("us"))
    ])
    .drop("forecast_datetime")
)
_summarize(f_wth_local_stats, "Forecast Weather Local Stats", 1)

After weather data is processed, let's go ahead to merge weather with the main training data.

> It's worth noting that **there exist overlappings of forecast weather data for those forecasts made nearby**. Hence `data_block_id` should be provided as another key to avoid errors when merging data. 

In [ ]:
train = (
    train
    .join(h_wth_local_stats, on=gp_keys_h, how="left")
    .join(f_wth_local_stats, on=["data_block_id", "datetime", "county"], how="left", suffix="_f")
)
_summarize(train, "Train with Weather", 1)

Then, you can save the processed data if you want!

In [ ]:
save_train_with_weather = True
if save_train_with_weather:
    train.write_parquet("train_with_weather.parquet")

<a id="st_dash"></a>
## 2. Towards Spatio-Temporal Analysis with Dash 
[**<span style="color:#FEF1FE; background-color:#535d70;border-radius: 5px; padding: 2px">Go to Table of Content</span>**](#toc)

Analyzing target sequences with weather data can provide hidden clues for advanced feature engineering and modeling. I've spent the past 2+ days to create a naive version of interactive data explorer based on `dash`. The main functionalities are summarized as follows,<br>
1. Target sequence plot
    * Select different counties with different target types.
    * Toggle on/off target sequences of specific `prediction_unit_id`.
2. Forecast weather plot
    * Select forecast weather feature, which is aggregated over stations in the same county.
    * Enable co-analyze with target sequences.
3. Estonia map with county boundary and central point
    * Select county to highlight where you're focusing now!

We can view this challenge as a 2-axis problem, namely **spatial** dimension and **temporal** one. For the former, prosumers in the **neighbor** area might be subject to the similar weather condition, which is an important factor affecting energy production. Hence, there might be **spatial dependency** among these prosumers. As for the latter, there exist **temporal patterns** within target/weather sequences, such as periodicity. Both can be helpful for featue engineering.

In [ ]:
def _get_colors(n_colors: int = 16) -> List[str]:
    """Reture color list for visualizing categorical column."""
    colors = plotly.colors.sample_colorscale("Bluered", [i / 15 for i in range(16)])
    
    # Visualize color palette
    fig = go.Figure()
    for i, c in enumerate(colors):
        fig.add_bar(x=[i], y=[1], marker_color=c, showlegend=False, name=c)
    fig.update_layout(
        title={"text": f"Color Palette of {n_colors} Colors", "yanchor": "top"},
        height=50,
        margin=dict(l=0, r=0, t=30, b=0),
        bargap=0
    )
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)
    fig.show()

    return colors

<a id="st_load_data"></a>
### *Load Necessary Data*
[**<span style="color:#FEF1FE; background-color:#535d70;border-radius: 5px; padding: 2px">Go to Table of Content</span>**](#toc)

In [ ]:
with open(PROC_DATA_PATH / "county_id2name.pkl", "rb") as f:
    county_id2name = pickle.load(f)
    county_name2id = {v: k for k, v in county_id2name.items()}
county_name2latlon = pl.read_parquet(PROC_DATA_PATH / "county_name2latlon.parquet")
train = train.with_columns(pl.col("county").cast(pl.Utf8).replace(county_id2name).alias("county_name"))

<a id="st_gen_opts"></a>
### *Generate Interactive Options*
[**<span style="color:#FEF1FE; background-color:#535d70;border-radius: 5px; padding: 2px">Go to Table of Content</span>**](#toc)

As described above, we need to specify options users can choose from.

In [ ]:
county_list = [f"{cid}-{name}" for cid, name in county_id2name.items()]
county_opts = [{"label": c, "value": c} for c in ["All"] + county_list]
county_opts_map = {c: [c] for c in county_list}
county_opts_map["All"] = county_list

# Colors for county central points
colors = _get_colors(16)
county_name2color = dict(zip((county_id2name.values()), colors))

In [ ]:
fwth_feat_list = [
    "temperature",
    "dewpoint",
    "cloudcover_high",
    "cloudcover_low",
    "cloudcover_mid",
    "cloudcover_total",
    "10_metre_u_wind_component",
    "10_metre_v_wind_component",
    "direct_solar_radiation",
    "surface_solar_radiation_downwards",
    "snowfall",
    "total_precipitation"
]
fwth_feat_opts = [{"label": c, "value": c} for c in fwth_feat_list]
fwth_feat_opts_map = {c: [c] for c in fwth_feat_list}

<a id="st_layout"></a>
### *Define Dashboard Layout*
[**<span style="color:#FEF1FE; background-color:#535d70;border-radius: 5px; padding: 2px">Go to Table of Content</span>**](#toc)

Next, we need to define the dashboard layout, which is the user interface.

In [ ]:
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

In [ ]:
app.layout = html.Div([
    dbc.Row([
        html.H1("Towards Spatio-Temporal Analysis", className="text-center")
    ]),

    dbc.Row([        
        dbc.Col([
            html.Div([
                html.H4("Select county:"),
                dcc.Dropdown(id="county-dropdown", options=county_opts, value="All", clearable=False)
            ])
        ], width=4),
        dbc.Col([
            html.Div([
                html.H4("Select target type:"),
                dbc.RadioItems(
                    id="target-radio",
                    options=[{"label": x, "value": x} for x in ["Consumption", "Production"]], 
                    value="Consumption"
                )
            ])
        ], width=4),
        dbc.Col([
            html.Div([
                html.H4("Select forecast weather feature:"),
                dcc.Dropdown(id="fwth-feat-dropdown", options=fwth_feat_opts, value=fwth_feat_list[0], clearable=False)
            ])
        ], width=4),

        html.Hr(),

        dbc.Row([
            dbc.Col([
                dcc.Graph(id="target"), 
                dcc.Graph(id="wth")
            ], width=6),
            dbc.Col([
                dcc.Graph(id="map") 
            ], width=6) 
        ]),      
    ]),
])

map_layout = go.Layout(
    autosize=True,
    hovermode="closest",
    mapbox_style="open-street-map",
    mapbox=dict(
        bearing=0,
        center=dict(lat=58.65, lon=24.95),
        pitch=0,
        zoom=6
    ),
    showlegend=False,
    margin=dict(l=5, r=20, t=20, b=5, pad=0)
)

<a id="st_cbs"></a>
### *Define Callbacks*
[**<span style="color:#FEF1FE; background-color:#535d70;border-radius: 5px; padding: 2px">Go to Table of Content</span>**](#toc)

Then, callbacks are defined to support output update (*e.g.,* figure rendering) based on user inputs.

In [ ]:
@callback(
    Output("target", "figure"),
    [Input("county-dropdown", "value"),
     Input("target-radio", "value")]
)
def update_target(slc_county, slc_target_type):
    """Update target sequences."""
    if slc_county == "All":
        slc_county_list = county_opts_map[county_list[0]] 
    else:
        slc_county_list = county_opts_map[slc_county]
    slc_county_name = ["-".join(c.split("-")[1:]) for c in slc_county_list]
    is_cons = 1 if slc_target_type == "Consumption" else 0
    target = (
        train 
        .filter((pl.col("county_name").is_in(slc_county_name)) & (pl.col("is_consumption") == is_cons))
        .sort("datetime")
    )
    fig = px.line(target, x="datetime", y="target", color="prediction_unit_id")
    fig.update_layout(
        margin=dict(l=0, r=0, t=30, b=0),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    
    return fig


@callback(
    Output("wth", "figure"),
    [Input("county-dropdown", "value"),
     Input("fwth-feat-dropdown", "value")]
)
def update_weather(slc_county, slc_fwth_feat):
    if slc_county == "All":
        slc_county_list = county_opts_map[county_list[0]] 
    else:
        slc_county_list = county_opts_map[slc_county]
    slc_county_name = ["-".join(c.split("-")[1:]) for c in slc_county_list]

    slc_fwth_feat_name = fwth_feat_opts_map[slc_fwth_feat][0]
    if f"{slc_fwth_feat_name}_local_mean_f" in train:
        feat_name_suffix = "local_mean_f"
    else:
        feat_name_suffix = "local_mean"
    slc_fwth_feat_name = f"{slc_fwth_feat_name}_{feat_name_suffix}"

    wth = (
        train
        .filter((pl.col("county_name").is_in(slc_county_name)))
        .select(["datetime", slc_fwth_feat_name])
        .unique()
        .sort("datetime")
    )
    fig = px.line(wth, x="datetime", y=slc_fwth_feat_name)
    fig.update_layout(
        margin=dict(l=0, r=0, t=30, b=0)
    )

    return fig


@callback(
    Output("map", "figure"),
    Input("county-dropdown", "value") 
)
def update_map(slc_county):
    slc_county_list = county_opts_map[slc_county]
    slc_county_name = ["-".join(c.split("-")[1:]) for c in slc_county_list]

    data = []
    # County boundary
    for r in county_name2latlon.drop_nulls().iter_rows(named=True):
        data.append(
            go.Scattermapbox(
                lat=r["lat_bound"],
                lon=r["lon_bound"],
                mode="lines",
                line=dict(width=1, color="gray"),
                name=r["county_name"],
                fill="toself" if r["county_name"] in slc_county_name else None,
            )
        )

    # County central point
    for county_name in sorted(slc_county_name):
        data.append(
            go.Scattermapbox(
                lat=county_name2latlon.filter(pl.col("county_name") == county_name)["lat"],
                lon=county_name2latlon.filter(pl.col("county_name") == county_name)["lon"],
                mode="markers",
                marker=dict(
                    size=10,
                    color=county_name2color[county_name],
                    opacity=0.7
                ),
                name=county_name,
                hoverinfo="name"
            )
        )

    return {
        "data": data,
        "layout": map_layout
    }

<a id="st_start"></a>
### *Start Exploration*
[**<span style="color:#FEF1FE; background-color:#535d70;border-radius: 5px; padding: 2px">Go to Table of Content</span>**](#toc)

Finally, we can start playing round with this dashboard! You can switch `jupyter_mode` to `"inline"` if you want to explore within the notebook. Owing to the `ngrok` session issue, I provide a quick demo below. Please feel free to deploy your own dashboard and play around with it!

In [ ]:
app.run(jupyter_mode="external", jupyter_server_url=tunnel.public_url)

![](https://media.giphy.com/media/v1.Y2lkPTc5MGI3NjExOHBjc2M3ZzdqdmNtdmE5bzV0ejlxMnZzZW8yZHJjaDd2MTI3c3IwaCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/X1NNUk8N3SSprG67Dn/source.gif)

<a id="baseline"></a>
## 3. Baseline Model with Local CV
[**<span style="color:#FEF1FE; background-color:#535d70;border-radius: 5px; padding: 2px">Go to Table of Content</span>**](#toc)

## Work in Progress...